# VacationPy Module-06-Challenge | python-api-challenge

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint


In [2]:
# Fix gmaps is_atomic failure
from collections.abc import *
from collections import abc


In [3]:
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_TD,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Albany,42.6001,-73.9662,94.57,58,81,1.99,US,1659985138
1,1,Pitimbu,-7.4706,-34.8086,81.19,72,30,15.75,BR,1659985499
2,2,Kapaa,22.0752,-159.3190,81.12,71,0,12.66,US,1659985501
3,3,Mestia,43.0458,42.7278,56.79,89,52,5.08,GE,1659985502
4,4,Bluff,-46.6000,168.3333,34.81,69,2,17.36,NZ,1659985504


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [7]:
# heatmap of humidity

city_data_df = city_data_df.dropna()


locations = city_data_df[["Lat", "Lng"]].astype(float)

humidity = city_data_df["Humidity"].astype(float)

fig = gmaps.figure()

# add heatmap layer
heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity,
    dissipating=False,
    max_intensity=300,
    point_radius=5
)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Filter cities by Max Temp between 70 and 80, Humidity less than 51%, and Cloudiness < 30%


ideal_cities_df = city_data_df[(city_data_df["Max Temp"]<81) & (city_data_df["Max Temp"]>70) & (city_data_df["Humidity"]<51) & (city_data_df["Cloudiness"]<30)]


ideal_cities_df

,City_TD,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,40,Kamenka,51.3223,42.7678,73.31,37,9,7.43,RU,1659985269
70,70,Atikokan,48.7555,-91.6216,75.42,34,0,6.55,CA,1659985604
154,154,Almaznyy,48.0448,40.0450,76.48,24,6,5.73,RU,1659985728
226,226,Manono,-7.3000,27.4167,78.66,26,2,5.68,CD,1659985842
242,242,Kalach-na-Donu,48.6910,43.5264,77.11,36,16,14.29,RU,1659985866
332,332,Stratoni,40.5141,23.8247,78.96,46,3,3.36,GR,1659986005
357,357,Moville,42.4889,-96.0725,76.69,27,0,8.97,US,1659986042
383,383,Krasnogvardeyskoye,45.8456,41.5186,74.89,40,0,7.11,RU,1659986080
423,423,Navoiy,40.0844,65.3792,74.68,25,0,2.30,UZ,1659986142
463,463,Lodja,-3.4833,23.4333,79.92,34,28,0.67,CD,1659986200


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Create Hotel DataFrame with empty "Hotel Name" column

hotel_df = ideal_cities_df[["City", "Lat", "Lng", "Country"]]

hotel_df["Hotel Name"] = ""

hotel_df

C:\Users\MattM\AppData\Local\Temp\ipykernel_16372\896114060.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Lat,Lng,Country,Hotel Name
40,Kamenka,51.3223,42.7678,RU,
70,Atikokan,48.7555,-91.6216,CA,
154,Almaznyy,48.0448,40.0450,RU,
226,Manono,-7.3000,27.4167,CD,
242,Kalach-na-Donu,48.6910,43.5264,RU,
332,Stratoni,40.5141,23.8247,GR,
357,Moville,42.4889,-96.0725,US,
383,Krasnogvardeyskoye,45.8456,41.5186,RU,
423,Navoiy,40.0844,65.3792,UZ,
463,Lodja,-3.4833,23.4333,CD,


In [11]:
#

baseURL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#search for the hotels

target_radius = 5000
target_type = "hotel"


# dictionary of parameters to update at each iteration 
parameters = {
    "type": target_type,
    "radius" :target_radius,
    "key": g_key
}

# use iterrows() to iterate through the rows in the DF
for index, row in hotel_df.iterrows():
    #grab the latitudes and longitudes from the columns in the DF
    latitude = row['Lat']
    longitude = row['Lng']
    
    # change location each iteration while leaving original params in place
    parameters["location"] = f"{latitude},{longitude}"
    
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_response = requests.get(base_url, params=parameters)
  

    # convert to json
    hotel_json = hotel_response.json()
    #pprint(hotel_json) # to check the indices of the hotel name 
    
 
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_json["results"][1]["name"]
        print(f"First hotel in {target_radius} meters is {hotel_json['results'][1]['name']}")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    print("--------------------------------------")

First hotel in 5000 meters is Kamenskaya Oosh
--------------------------------------
First hotel in 5000 meters is Avenue A Realty Ltd Brokerage
--------------------------------------
First hotel in 5000 meters is Kurskaya Oosh
--------------------------------------
First hotel in 5000 meters is Salle du Royaume des Témoins de Jéhovah
--------------------------------------
First hotel in 5000 meters is Hotel Laguna
--------------------------------------
First hotel in 5000 meters is Μεταλλευτικές Εγκαταστάσεις Στρατωνίου / Stratoni Mining Facilities
--------------------------------------
First hotel in 5000 meters is Shazam Atm
--------------------------------------
First hotel in 5000 meters is Zags
--------------------------------------
First hotel in 5000 meters is Silk Road Hotel Termez
--------------------------------------
First hotel in 5000 meters is Lodja, Republica Democrática do Congo
--------------------------------------
First hotel in 5000 meters is Budget House
---------

In [12]:
# checking the DF with the hotel names in the new column
hotel_df.head()

,City,Lat,Lng,Country,Hotel Name
40,Kamenka,51.3223,42.7678,RU,Kamenskaya Oosh
70,Atikokan,48.7555,-91.6216,CA,Avenue A Realty Ltd Brokerage
154,Almaznyy,48.0448,40.0450,RU,Kurskaya Oosh
226,Manono,-7.3000,27.4167,CD,Salle du Royaume des Témoins de Jéhovah
242,Kalach-na-Donu,48.6910,43.5264,RU,Hotel Laguna


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))